# Traditional Sector Rotation Strategy

## Overview
This notebook implements traditional sector rotation strategies using ratio-based mean reversion signals.

### Strategy Focus:
- **Technology vs Utilities (XLK/XLU)**: Growth vs Defensive rotation
- **Financials vs REITs (XLF/XLRE)**: Rate sensitivity plays

### Key Concepts:
- **Risk-On/Risk-Off Cycles**: Tech outperforms during growth, Utilities during uncertainty
- **Interest Rate Sensitivity**: Financials benefit from rising rates, REITs suffer
- **Mean Reversion**: Sector leadership rotates cyclically
- **Z-Score Signals**: Statistical thresholds for rotation timing

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

print("📊 Traditional Sector Rotation Strategy Environment Ready")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("🎯 Strategy: Growth vs Defensive + Rate Sensitivity Rotations")

## 1. Sector ETF Data Collection

In [4]:
# Define traditional sector rotation pairs
TRADITIONAL_SECTORS = {
    # Growth vs Defensive
    'XLK': 'XLK',    # Technology Select Sector SPDR
    'XLU': 'XLU',    # Utilities Select Sector SPDR
    
    # Rate Sensitivity
    'XLF': 'XLF',    # Financial Select Sector SPDR
    'XLRE': 'XLRE',  # Real Estate Select Sector SPDR
    
    # Additional context ETFs
    'SPY': 'SPY',    # S&P 500 (benchmark)
    'TLT': 'TLT',    # 20+ Year Treasury (rates)
    'VIX': '^VIX'    # Volatility index
}

def fetch_sector_data(etfs_dict, period='5y'):
    """Fetch sector ETF data with error handling"""
    print("🔄 Fetching sector ETF data...")
    
    sector_data = {}
    for etf_name, symbol in etfs_dict.items():
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period, interval='1d')
            if not data.empty and len(data) > 100:
                sector_data[etf_name] = data['Close']
                print(f"  ✅ {etf_name}: {len(data)} days")
            else:
                print(f"  ❌ {etf_name}: Insufficient data ({len(data) if not data.empty else 0} days)")
        except Exception as e:
            print(f"  ❌ {etf_name}: Failed to fetch - {str(e)}")
    
    if sector_data:
        df = pd.DataFrame(sector_data).dropna()
        if len(df) > 0:  # Check if dataframe has data
            print(f"\n✅ Sector dataset: {len(df.columns)} ETFs, {len(df)} observations")
            print(f"📊 Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
            return df
        else:
            print(f"\n❌ No data available after alignment and cleaning")
    else:
        print(f"\n❌ No sector data could be fetched")
    
    return pd.DataFrame()

# Fetch sector data
sector_prices = fetch_sector_data(TRADITIONAL_SECTORS)

if not sector_prices.empty:
    print(f"\n🏭 Available Sector ETFs: {list(sector_prices.columns)}")
    sector_prices.head()
else:
    print(f"\n⚠️ No sector data available - please check internet connection or try again later")

🔄 Fetching sector ETF data...


$XLK: possibly delisted; no price data found  (period=5y)


  ❌ XLK: Insufficient data (0 days)
  ✅ XLU: 1256 days
  ✅ XLF: 1256 days
  ✅ XLRE: 1256 days
  ✅ SPY: 1256 days
  ✅ TLT: 1256 days
  ✅ VIX: 1256 days

❌ No data available after alignment and cleaning

⚠️ No sector data available - please check internet connection or try again later


## 2. Sector Rotation Strategy Implementation

In [ ]:
def calculate_sector_ratio(price_data, etf1, etf2):
    """Calculate sector ratio between two ETFs"""
    ratio = price_data[etf1] / price_data[etf2]
    return ratio

def calculate_sector_zscore(ratio, lookback_window=60):
    """Calculate rolling z-score of sector ratio"""
    rolling_mean = ratio.rolling(window=lookback_window).mean()
    rolling_std = ratio.rolling(window=lookback_window).std()
    z_score = (ratio - rolling_mean) / rolling_std
    return z_score, rolling_mean, rolling_std

def generate_sector_rotation_signals(price_data, etf1, etf2, 
                                   entry_threshold=2.0, exit_threshold=0.5, lookback_window=60):
    """Generate sector rotation signals based on z-scored ratios"""
    
    # Calculate ratio and z-score
    ratio = calculate_sector_ratio(price_data, etf1, etf2)
    z_score, rolling_mean, rolling_std = calculate_sector_zscore(ratio, lookback_window)
    
    # Create signals DataFrame
    signals = pd.DataFrame(index=price_data.index)
    signals['ratio'] = ratio
    signals['z_score'] = z_score
    signals['rolling_mean'] = rolling_mean
    signals['rolling_std'] = rolling_std
    
    # Sector rotation signals
    signals['rotate_to_etf1'] = z_score < -entry_threshold   # Ratio low, rotate to ETF1
    signals['rotate_to_etf2'] = z_score > entry_threshold    # Ratio high, rotate to ETF2
    
    # Neutral/exit signals
    signals['neutral_signal'] = abs(z_score) < exit_threshold
    
    # Position logic for sector allocation
    signals['sector_allocation'] = 0  # 0 = neutral, 1 = favor ETF1, -1 = favor ETF2
    current_allocation = 0
    
    for i in range(len(signals)):
        if current_allocation == 0:  # Neutral allocation
            if signals['rotate_to_etf1'].iloc[i]:
                current_allocation = 1  # Rotate to ETF1
            elif signals['rotate_to_etf2'].iloc[i]:
                current_allocation = -1  # Rotate to ETF2
        elif current_allocation != 0:  # Have sector bias
            if signals['neutral_signal'].iloc[i]:
                current_allocation = 0  # Back to neutral
        
        signals['sector_allocation'].iloc[i] = current_allocation
    
    # Calculate strategy returns
    signals['etf1_return'] = price_data[etf1].pct_change()
    signals['etf2_return'] = price_data[etf2].pct_change()
    
    # Strategy return based on allocation
    strategy_returns = []
    for i in range(len(signals)):
        allocation = signals['sector_allocation'].iloc[i]
        etf1_ret = signals['etf1_return'].iloc[i]
        etf2_ret = signals['etf2_return'].iloc[i]
        
        if allocation == 1:  # Favor ETF1
            strategy_ret = etf1_ret
        elif allocation == -1:  # Favor ETF2
            strategy_ret = etf2_ret
        else:  # Neutral - equal weight
            strategy_ret = 0.5 * etf1_ret + 0.5 * etf2_ret
        
        strategy_returns.append(strategy_ret if not np.isnan(strategy_ret) else 0)
    
    signals['strategy_return'] = strategy_returns
    signals['cumulative_return'] = (1 + pd.Series(strategy_returns, index=signals.index)).cumprod()
    
    # Buy and hold benchmarks
    signals['etf1_cumulative'] = (1 + signals['etf1_return'].fillna(0)).cumprod()
    signals['etf2_cumulative'] = (1 + signals['etf2_return'].fillna(0)).cumprod()
    signals['equal_weight_cumulative'] = (1 + (0.5 * signals['etf1_return'] + 0.5 * signals['etf2_return']).fillna(0)).cumprod()
    
    return signals

print("✅ Sector rotation strategy functions implemented")

## 3. Test Traditional Sector Rotation Strategies

In [ ]:
# Test traditional sector rotation strategies
print("🎯 Testing Traditional Sector Rotation Strategies\n")

# Define sector rotation pairs with descriptions
ROTATION_PAIRS = [
    ('XLK', 'XLU', 'Technology vs Utilities (Growth vs Defensive)'),
    ('XLF', 'XLRE', 'Financials vs REITs (Rate Sensitivity)')
]

rotation_results = []

for etf1, etf2, description in ROTATION_PAIRS:
    if etf1 in sector_prices.columns and etf2 in sector_prices.columns:
        try:
            print(f"📊 Analyzing {description}...")
            
            # Generate rotation signals
            signals = generate_sector_rotation_signals(sector_prices, etf1, etf2)
            
            # Calculate performance metrics
            strategy_cumulative = signals['cumulative_return']
            total_return = strategy_cumulative.iloc[-1] - 1
            
            # Benchmark returns
            etf1_total = signals['etf1_cumulative'].iloc[-1] - 1
            etf2_total = signals['etf2_cumulative'].iloc[-1] - 1
            equal_weight_total = signals['equal_weight_cumulative'].iloc[-1] - 1
            
            # Risk metrics
            volatility = signals['strategy_return'].std() * np.sqrt(252)
            sharpe_ratio = (total_return * 252 / len(signals)) / volatility if volatility > 0 else 0
            
            max_drawdown = ((strategy_cumulative / strategy_cumulative.expanding().max()) - 1).min()
            
            # Rotation statistics
            rotations = signals['sector_allocation'].diff().abs().sum() / 2
            time_in_etf1 = (signals['sector_allocation'] == 1).mean()
            time_in_etf2 = (signals['sector_allocation'] == -1).mean()
            time_neutral = (signals['sector_allocation'] == 0).mean()
            
            # Current status
            current_ratio = signals['ratio'].iloc[-1]
            current_zscore = signals['z_score'].iloc[-1]
            current_allocation = signals['sector_allocation'].iloc[-1]
            
            rotation_results.append({
                'strategy': description,
                'etf1': etf1,
                'etf2': etf2,
                'strategy_return': total_return,
                'etf1_return': etf1_total,
                'etf2_return': etf2_total,
                'equal_weight_return': equal_weight_total,
                'outperformance': total_return - equal_weight_total,
                'sharpe_ratio': sharpe_ratio,
                'max_drawdown': max_drawdown,
                'volatility': volatility,
                'num_rotations': rotations,
                'time_in_etf1': time_in_etf1,
                'time_in_etf2': time_in_etf2,
                'time_neutral': time_neutral,
                'current_ratio': current_ratio,
                'current_zscore': current_zscore,
                'current_allocation': current_allocation
            })
            
            print(f"  Strategy Return: {total_return:.2%}")
            print(f"  {etf1} Return: {etf1_total:.2%}")
            print(f"  {etf2} Return: {etf2_total:.2%}")
            print(f"  Equal Weight: {equal_weight_total:.2%}")
            print(f"  Outperformance: {total_return - equal_weight_total:.2%}")
            print(f"  Sharpe Ratio: {sharpe_ratio:.3f}")
            print(f"  Current Z-Score: {current_zscore:.2f}")
            print()
            
        except Exception as e:
            print(f"  ❌ Error: {str(e)}\n")

# Results analysis
if rotation_results:
    rotation_df = pd.DataFrame(rotation_results)
    
    print(f"📊 Traditional Sector Rotation Results Summary:")
    display(rotation_df[['strategy', 'strategy_return', 'outperformance', 'sharpe_ratio', 'max_drawdown', 'num_rotations']].round(3))
    
    # Best performing strategy
    best_strategy = rotation_df.loc[rotation_df['sharpe_ratio'].idxmax()]
    print(f"\n🏆 Best Strategy: {best_strategy['strategy']}")
    print(f"   Sharpe Ratio: {best_strategy['sharpe_ratio']:.3f}")
    print(f"   Outperformance: {best_strategy['outperformance']:.2%}")
    
else:
    print("❌ No sector rotation results generated")

## 4. Detailed Analysis and Visualization

In [ ]:
def plot_sector_rotation_analysis(signals, etf1, etf2, strategy_name):
    """Comprehensive visualization of sector rotation strategy"""
    
    fig, axes = plt.subplots(5, 1, figsize=(15, 20))
    
    # 1. Sector Ratio with Bands
    ax1 = axes[0]
    ax1.plot(signals.index, signals['ratio'], label=f'{etf1}/{etf2} Ratio', color='blue', linewidth=1.5)
    ax1.plot(signals.index, signals['rolling_mean'], label='Rolling Mean', color='red', linestyle='--')
    ax1.fill_between(signals.index, 
                     signals['rolling_mean'] + 2*signals['rolling_std'],
                     signals['rolling_mean'] - 2*signals['rolling_std'],
                     alpha=0.2, color='gray', label='±2σ Band')
    ax1.set_title(f'{strategy_name}: {etf1}/{etf2} Ratio Analysis', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Ratio Value')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Z-Score with Rotation Thresholds
    ax2 = axes[1]
    ax2.plot(signals.index, signals['z_score'], label='Z-Score', color='purple', linewidth=1.5)
    ax2.axhline(y=2, color='red', linestyle='--', label=f'Rotate to {etf2} (+2σ)')
    ax2.axhline(y=-2, color='green', linestyle='--', label=f'Rotate to {etf1} (-2σ)')
    ax2.axhline(y=0.5, color='orange', linestyle=':', label='Neutral Threshold')
    ax2.axhline(y=-0.5, color='orange', linestyle=':')
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax2.set_title('Z-Score with Sector Rotation Thresholds', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Z-Score')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Sector Allocation Over Time
    ax3 = axes[2]
    allocation_changes = signals['sector_allocation'].diff().fillna(0)
    
    # Color coding for allocations
    colors = {'1': 'green', '-1': 'red', '0': 'gray'}
    for allocation in [-1, 0, 1]:
        mask = signals['sector_allocation'] == allocation
        if allocation == 1:
            label = f'Favor {etf1}'
            color = 'green'
        elif allocation == -1:
            label = f'Favor {etf2}'
            color = 'red'
        else:
            label = 'Neutral'
            color = 'gray'
        
        ax3.fill_between(signals.index, 0, signals['sector_allocation'], 
                        where=mask, alpha=0.7, color=color, label=label)
    
    ax3.set_title('Sector Allocation Over Time', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Sector Allocation')
    ax3.set_ylim(-1.5, 1.5)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Individual ETF Performance
    ax4 = axes[3]
    ax4.plot(signals.index, signals['etf1_cumulative'], label=f'{etf1}', color='blue', linewidth=2)
    ax4.plot(signals.index, signals['etf2_cumulative'], label=f'{etf2}', color='orange', linewidth=2)
    ax4.plot(signals.index, signals['equal_weight_cumulative'], label='Equal Weight', color='gray', linestyle='--', linewidth=2)
    ax4.set_title('Individual Sector Performance', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Cumulative Return')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Strategy vs Benchmark Performance
    ax5 = axes[4]
    ax5.plot(signals.index, signals['cumulative_return'], label='Rotation Strategy', color='darkgreen', linewidth=3)
    ax5.plot(signals.index, signals['equal_weight_cumulative'], label='Equal Weight Benchmark', color='black', linestyle='--', linewidth=2)
    ax5.set_title(f'{strategy_name} Performance vs Benchmark', fontsize=14, fontweight='bold')
    ax5.set_ylabel('Cumulative Return')
    ax5.set_xlabel('Date')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Generate detailed analysis for each strategy
if 'rotation_results' in locals() and rotation_results:
    for result in rotation_results:
        etf1, etf2 = result['etf1'], result['etf2']
        strategy_name = result['strategy']
        
        print(f"\n📊 Detailed Analysis: {strategy_name}")
        
        # Generate signals for visualization
        signals = generate_sector_rotation_signals(sector_prices, etf1, etf2)
        
        # Plot analysis
        plot_sector_rotation_analysis(signals, etf1, etf2, strategy_name)
        
        # Performance metrics
        print(f"\n📈 Performance Metrics:")
        print(f"   Strategy Return: {result['strategy_return']:.2%}")
        print(f"   Outperformance: {result['outperformance']:.2%}")
        print(f"   Sharpe Ratio: {result['sharpe_ratio']:.3f}")
        print(f"   Max Drawdown: {result['max_drawdown']:.2%}")
        print(f"   Volatility: {result['volatility']:.2%}")
        print(f"   Number of Rotations: {result['num_rotations']:.0f}")
        
        print(f"\n⏱️ Time Allocation:")
        print(f"   Time in {etf1}: {result['time_in_etf1']:.1%}")
        print(f"   Time in {etf2}: {result['time_in_etf2']:.1%}")
        print(f"   Time Neutral: {result['time_neutral']:.1%}")
        
        print(f"\n📍 Current Status:")
        current_allocation = result['current_allocation']
        if current_allocation == 1:
            allocation_text = f"Favor {etf1}"
        elif current_allocation == -1:
            allocation_text = f"Favor {etf2}"
        else:
            allocation_text = "Neutral"
        
        print(f"   Current Allocation: {allocation_text}")
        print(f"   Current Z-Score: {result['current_zscore']:.2f}")
        print("\n" + "="*80)

## 5. Strategy Optimization

In [ ]:
def optimize_sector_rotation_strategy(price_data, etf1, etf2, strategy_name):
    """Optimize sector rotation strategy parameters"""
    print(f"🔧 Optimizing {strategy_name} Strategy...\n")
    
    # Parameter ranges for optimization
    entry_thresholds = [1.5, 2.0, 2.5, 3.0]
    exit_thresholds = [0.2, 0.5, 0.8, 1.0]
    lookback_windows = [30, 60, 90, 120]
    
    optimization_results = []
    
    for entry_thresh in entry_thresholds:
        for exit_thresh in exit_thresholds:
            for lookback in lookback_windows:
                try:
                    # Generate signals with test parameters
                    test_signals = generate_sector_rotation_signals(
                        price_data, etf1, etf2,
                        entry_threshold=entry_thresh,
                        exit_threshold=exit_thresh,
                        lookback_window=lookback
                    )
                    
                    # Calculate performance metrics
                    strategy_return = test_signals['cumulative_return'].iloc[-1] - 1
                    equal_weight_return = test_signals['equal_weight_cumulative'].iloc[-1] - 1
                    outperformance = strategy_return - equal_weight_return
                    
                    volatility = test_signals['strategy_return'].std() * np.sqrt(252)
                    sharpe_ratio = (strategy_return * 252 / len(test_signals)) / volatility if volatility > 0 else 0
                    
                    max_drawdown = ((test_signals['cumulative_return'] / test_signals['cumulative_return'].expanding().max()) - 1).min()
                    num_rotations = test_signals['sector_allocation'].diff().abs().sum() / 2
                    
                    optimization_results.append({
                        'entry_threshold': entry_thresh,
                        'exit_threshold': exit_thresh,
                        'lookback_window': lookback,
                        'strategy_return': strategy_return,
                        'outperformance': outperformance,
                        'sharpe_ratio': sharpe_ratio,
                        'max_drawdown': max_drawdown,
                        'num_rotations': num_rotations
                    })
                    
                except Exception as e:
                    continue
    
    if optimization_results:
        opt_df = pd.DataFrame(optimization_results)
        
        # Find best parameters by Sharpe ratio
        best_params = opt_df.loc[opt_df['sharpe_ratio'].idxmax()]
        
        print("🏆 Optimal Parameters (by Sharpe Ratio):")
        print(f"   Entry Threshold: {best_params['entry_threshold']}")
        print(f"   Exit Threshold: {best_params['exit_threshold']}")
        print(f"   Lookback Window: {best_params['lookback_window']} days")
        print(f"   Sharpe Ratio: {best_params['sharpe_ratio']:.3f}")
        print(f"   Strategy Return: {best_params['strategy_return']:.2%}")
        print(f"   Outperformance: {best_params['outperformance']:.2%}")
        
        # Show top parameter combinations
        print(f"\n📊 Top 10 Parameter Combinations:")
        top_params = opt_df.nlargest(10, 'sharpe_ratio')[['entry_threshold', 'exit_threshold', 'lookback_window', 'sharpe_ratio', 'outperformance']]
        display(top_params.round(3))
        
        return best_params, opt_df
    
    return None, None

# Optimize each strategy
optimization_summary = []

if 'rotation_results' in locals() and rotation_results:
    for result in rotation_results:
        etf1, etf2 = result['etf1'], result['etf2']
        strategy_name = result['strategy']
        
        best_params, opt_df = optimize_sector_rotation_strategy(sector_prices, etf1, etf2, strategy_name)
        
        if best_params is not None:
            optimization_summary.append({
                'strategy': strategy_name,
                'etf1': etf1,
                'etf2': etf2,
                'best_entry_threshold': best_params['entry_threshold'],
                'best_exit_threshold': best_params['exit_threshold'],
                'best_lookback': best_params['lookback_window'],
                'optimized_sharpe': best_params['sharpe_ratio'],
                'optimized_return': best_params['strategy_return'],
                'optimized_outperformance': best_params['outperformance']
            })
        
        print("\n" + "="*80 + "\n")
    
    # Summary of optimized strategies
    if optimization_summary:
        opt_summary_df = pd.DataFrame(optimization_summary)
        print("📊 Optimization Summary for All Strategies:")
        display(opt_summary_df.round(3))

## 6. Current Trading Signals

In [ ]:
def get_current_sector_signals(rotation_results):
    """Get current sector rotation signals"""
    print("🎯 Current Traditional Sector Rotation Signals\n")
    
    current_signals = []
    
    for result in rotation_results:
        strategy_name = result['strategy']
        current_zscore = result['current_zscore']
        current_allocation = result['current_allocation']
        etf1, etf2 = result['etf1'], result['etf2']
        
        # Determine signal strength and recommendation
        if current_zscore < -2.0:
            signal = f"ROTATE TO {etf1}"
            strength = "Strong" if current_zscore < -2.5 else "Moderate"
            reasoning = f"{etf1} underperforming, expect catch-up"
        elif current_zscore > 2.0:
            signal = f"ROTATE TO {etf2}"
            strength = "Strong" if current_zscore > 2.5 else "Moderate"
            reasoning = f"{etf2} underperforming, expect catch-up"
        elif abs(current_zscore) < 0.5:
            signal = "NEUTRAL"
            strength = "Equal Weight"
            reasoning = "Sectors fairly valued relative to each other"
        else:
            signal = "HOLD"
            strength = "Current Position"
            reasoning = "Maintain current allocation"
        
        current_signals.append({
            'strategy': strategy_name,
            'signal': signal,
            'strength': strength,
            'z_score': current_zscore,
            'current_allocation': current_allocation,
            'reasoning': reasoning,
            'sharpe_ratio': result['sharpe_ratio']
        })
        
        print(f"{strategy_name}:")
        print(f"   Signal: {signal} ({strength})")
        print(f"   Z-Score: {current_zscore:.2f}")
        print(f"   Reasoning: {reasoning}")
        print(f"   Historical Sharpe: {result['sharpe_ratio']:.3f}")
        print()
    
    return pd.DataFrame(current_signals)

# Get current signals
if 'rotation_results' in locals() and rotation_results:
    current_sector_signals = get_current_sector_signals(rotation_results)
    
    print(f"📊 Current Signal Summary:")
    display(current_sector_signals[['strategy', 'signal', 'strength', 'z_score', 'sharpe_ratio']].round(3))
    
    # Active rotation opportunities
    active_rotations = current_sector_signals[current_sector_signals['signal'].str.contains('ROTATE')]
    if not active_rotations.empty:
        print(f"\n🚨 Active Rotation Opportunities:")
        for _, signal in active_rotations.iterrows():
            print(f"   {signal['strategy']}: {signal['signal']} ({signal['strength']})")
            print(f"     Z-Score: {signal['z_score']:.2f}")
            print(f"     {signal['reasoning']}")
    else:
        print(f"\n⏳ No active rotation signals - Hold current allocations or maintain neutral")
        
else:
    print("❌ No sector rotation analysis available")

## 7. Strategy Summary

In [ ]:
print("📋 Traditional Sector Rotation Strategy Summary")
print("=" * 60)

if 'rotation_results' in locals() and rotation_results:
    print(f"\n🔬 Strategy Performance Analysis:")
    total_strategies = len(rotation_results)
    profitable_strategies = sum(1 for r in rotation_results if r['outperformance'] > 0)
    avg_outperformance = np.mean([r['outperformance'] for r in rotation_results])
    best_sharpe = max(r['sharpe_ratio'] for r in rotation_results)
    
    print(f"   Strategies tested: {total_strategies}")
    print(f"   Outperforming strategies: {profitable_strategies}")
    print(f"   Success rate: {profitable_strategies/total_strategies:.1%}")
    print(f"   Average outperformance: {avg_outperformance:.2%}")
    print(f"   Best Sharpe ratio: {best_sharpe:.3f}")

print(f"\n💡 Traditional Sector Rotation Insights:")
print(f"   • Growth vs Defensive rotations capture risk sentiment cycles")
print(f"   • Rate sensitivity trades benefit from Fed policy changes")
print(f"   • Mean reversion provides systematic rotation opportunities")
print(f"   • Z-score thresholds offer objective timing signals")
print(f"   • Outperforms static allocation through tactical timing")

print(f"\n🎯 Implementation Advantages:")
print(f"   • Liquid ETF execution with tight spreads")
print(f"   • Clear risk management through position sizing")
print(f"   • Transparent signals based on statistical measures")
print(f"   • Adaptable to changing market regimes")
print(f"   • Can be enhanced with macro economic filters")

print(f"\n🚀 Strategy Applications:")
print(f"   • Core satellite portfolio construction")
print(f"   • Tactical asset allocation overlay")
print(f"   • Risk parity strategy enhancement")
print(f"   • Structured products with sector themes")

print(f"\n🌟 Key Strategy Components:")
print(f"   • Technology vs Utilities (XLK/XLU) - Growth vs Defensive")
print(f"   • Financials vs REITs (XLF/XLRE) - Rate sensitivity")
print(f"   • Z-score normalization for consistent signals")
print(f"   • Parameter optimization for enhanced performance")

print(f"\n✅ Traditional Sector Rotation Strategy Complete")